## Задание 1

В файле содержится информация о покупках людей:

id | Товар | Количество
---| ------------ | ---------:
 1 | Апельсины    | 5
 1 | Яблоки       | 120
 1 | Груши        | 25
 2 | Арбуз        | 4
 2 | Ананас       | 1
 
 
  * id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)  
  * Товар – наименование товара  
  * Количество – число единиц купленного товара  
  
Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.
    
    
1_Товар | 2_Товар | Встречаемость
------| ------------ | ---------:
 Чай  | Арбуз    | 5
 Арбуз  | Сгущёнка       | 3
 Чай | Сгущёнка        | 1

 
 * 1_Товар – наименование первого товара  
 * 2_Товар – наименование второго товара  
 * Встречаемость – число раз, когда такая пара была встречена  
 
Другими словами: 2 раза люди покупали одновременно чай и арбуз, 1 раз одновременно покупали арбуз и сгущёнку и 1 раз одновременно были куплены чай со сгущёнкой.

Напишите код на python для получения нужной таблицы и укажите 5 наиболее распространённых паттернов.

In [130]:
# Загркзим необходимые библиотеки 

import pandas as pd
import numpy as np
import matplotlib as plt
from itertools import*  

### Подготовка данных

In [131]:
data = pd.read_csv('test1_completed.csv') # считаем данные 

In [132]:
data.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [133]:
data.shape # посмотрим на размер данных

(43514, 3)

In [134]:
data.nunique() # посмотрим на уникальные значения каждого столбца

id            3273
Товар          199
Количество     101
dtype: int64

In [135]:
data.drop(columns = 'Количество', inplace = True) # удалим ненужную колонку 
data.rename(columns = {'Товар' : 'goods'}, inplace = True) # переименуем колонку для более удобного обращения 

### Предварительный расчёт 

Необходимо узнать сколько существует всевозможных пар товаров, исключая повторения ([лимон, яблоко] и [яблоко, лимон]), в предоставленных данных.

In [136]:
data.goods.nunique() # всего уникальных продуктов в предоставленных данных

199

Общая формула, которая позволяет найти число сочетаний из n объектов по r имеет вид:

$$C_r = {n ! \over {n!(n-r)}!}$$ 

где:  
$С_r$- number of combinations;  
$n$ - total number of objects in the set;  
$r$ - number of choosing objects from yhe set;

Подставив числа в формулу - получаем 19 701 сочетаний может быть в наших данных. 

### Работа над данными

Последовательность действий:  
    
   1. Для каждого id найдем товары, которые он купил;  
   2. Составим все возможные пары продуктов и будем добавлять их в словарь как ключи со значением: 1;  
   3. Если такой ключ уже будет в словаре - прибавляем единицу к значению;  
   4. Поставим условие, чтобы убрать повторяющиеся комбинации([лимон, яблоко] и [яблоко, лимон]) из словаря;  

In [137]:
empty_dict = {} # пустой словарь

for ids in data.id.unique():
    staff = data.query('id == @ids').goods # для каждого id узнаем приобретённый товар 
    for pair in permutations(staff, 2): # создадим всевозможные пары товаров 
        if pair not in empty_dict: # и для каждой пары проверим её наличие в словаре 
            split_pair = pair[1],pair[0] # поменяем местами товары в паре 
            if split_pair not in empty_dict: # и проверим нет ли такой пары в списке 
                empty_dict[pair] = 1
        else: 
            empty_dict[pair] = empty_dict[pair] + 1 #если какая-то пара уже есть в спике: добавим единичку к её значению

In [138]:
len(empty_dict) # проверим число пар у нас в словаре

19697

Число пар не превышает максимально допустимое число сочетаний - ошибки нет.

### Окончательный результат

In [139]:
result = pd.DataFrame.from_dict(empty_dict, orient = 'index') \
            .reset_index() \
            .rename(columns= {'index' : 'goods', 0 : 'Встречаемость'}) # переведём словарь в DataFrame

In [140]:
product_1 = []
product_2 = []

for product in result.goods:
    product_1.append(product[0])
    product_2.append(product[1])
    
result.drop(columns = 'goods') # удалим ненужную колонку 
result['1_Товар'] = product_1 # добавим колонку с первым товаром 
result['2_Товар'] = product_2 # добавим колонку со вторым товаром 
result = result[['1_Товар', '2_Товар','Встречаемость']] #установим правильный порядок столбцов     

In [141]:
# посмотрим на результат 
result.sort_values('Встречаемость', ascending = False).head(5)

,1_Товар,2_Товар,Встречаемость
291,Огурцы Луховицкие,Укроп,431
297,Петрушка,Укроп,408
3630,Арбуз,Огурцы Луховицкие,345
855,Кабачки,Огурцы Луховицкие,326
443,Кинза,Укроп,303
